In [9]:
import pandas as pd

# combining all datasets from NBA Data
reg_season_data = pd.concat([pd.read_csv("NBA-Data-2010-2024/regular_season_box_scores_2010_2024_part_1.csv"), pd.read_csv("NBA-Data-2010-2024/regular_season_box_scores_2010_2024_part_2.csv"), pd.read_csv("NBA-Data-2010-2024/regular_season_box_scores_2010_2024_part_3.csv")])

# view first 5 entries
# already in player numeric order

reg_season_data.shape
reg_season_data.iloc[0:5, 0:15]



,season_year,game_date,gameId,matchup,teamId,teamCity,teamName,teamTricode,teamSlug,personId,personName,position,comment,jerseyNum,minutes
0,2010-11,2010-11-10,21000112,NJN @ CLE,1610612751,New Jersey,Nets,NJN,nets,693,Joe Smith,NaN,NaN,NaN,4:21
1,2010-11,2010-11-17,21000165,NJN @ UTA,1610612751,New Jersey,Nets,NJN,nets,693,Joe Smith,NaN,DNP - Coach's Decision,NaN,NaN
2,2010-11,2010-11-27,21000237,NJN @ PHI,1610612751,New Jersey,Nets,NJN,nets,693,Joe Smith,NaN,DNP - Coach's Decision,NaN,NaN
3,2010-11,2010-12-12,21000351,NJN vs. LAL,1610612751,New Jersey,Nets,NJN,nets,693,Joe Smith,NaN,DNP - Coach's Decision,NaN,NaN
4,2010-11,2010-10-29,21000020,NJN vs. SAC,1610612751,New Jersey,Nets,NJN,nets,693,Joe Smith,F,NaN,NaN,9:53
